Copy a CustomVision.ai Project including images and tags to a new Project. For the purposes of DR, use a Training Key of an instance of Custom Vision in a seperate Resource Group, or Subscription. For the purposes of HA, consider use of a Training Key in a different Region.

In [ ]:
import argparse, os, sys, time

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateBatch, ImageUrlCreateEntry, Region

In [ ]:
def migrate_tags(source_trainer, destination_trainer, source_project_id, destination_project_id):
    tags =  source_trainer.get_tags(source_project_id)
    print('Found', len(tags), 'tags...')

    created_tags = {}
    
    for tag in source_trainer.get_tags(source_project_id):
        print('Creating tag', tag.name, tag.id)
        created_tags[tag.id] = destination_trainer.create_tag(destination_project_id, tag.name, description=tag.description, type=tag.type).id
    return created_tags

In [ ]:
def migrate_images(source_trainer, destination_trainer, source_project_id, destination_project_id, created_tags):
    # Tagged images...
    count = source_trainer.get_tagged_image_count(source_project_id)
    print ('Found:', count, 'tagged images...')

    migrated = 0
    
    while(count > 0):
        count_to_migrate = min(count, 50)
        print ('Getting', count_to_migrate, 'images...')
        images = source_trainer.get_tagged_images(source_project_id, take=count_to_migrate, skip=migrated)
        images_to_upload = []
        for image in images:
            print ('Migrating', image.id, image.original_image_uri)
            if image.regions:
                regions = []
                for region in image.regions:
                    print ('Found region:', region.region_id, region.tag_id, region.left, region.top, region.width, region.height)
                    regions.append(Region(tag_id=created_tags[region.tag_id], left=region.left, top=region.top, width=region.width, height=region.height))
                entry = ImageUrlCreateEntry(url=image.original_image_uri, regions=regions)
            else:
                tag_ids = []
                for tag in image.tags:
                    print ('Found tag:', tag.tag_name, tag.tag_id)
                    tag_ids.append(created_tags[tag.tag_id])
                entry = ImageUrlCreateEntry(url=image.original_image_uri, tag_ids=tag_ids)

            images_to_upload.append(entry)

        upload_result = destination_trainer.create_images_from_urls(destination_project_id, images=images_to_upload)

        for image in upload_result.images:
            print ('Image status:', image.status)
        exit(-1)

        migrated += count_to_migrate
        count -= count_to_migrate

    # Untagged images...
    count = source_trainer.get_untagged_image_count(source_project_id)
    print ('Found:', count, 'untagged images...')
    
    migrated = 0
    
    while(count > 0):
        count_to_migrate = min(count, 50)
        print ('Getting', count_to_migrate, 'images...')
        images = source_trainer.get_untagged_images(source_project_id, take=count_to_migrate, skip=migrated)
        images_to_upload = []
        for image in images:
            print ("Migrating", image.id, image.original_image_uri)
            images_to_upload.append(ImageUrlCreateEntry(url=image.original_image_uri))

        upload_result = destination_trainer.create_images_from_urls(destination_project_id, images=images_to_upload)
        
        for image in upload_result.images:
            print ('Image status:', image.status)
        exit(-1)
            
        migrated += count_to_migrate
        count -= count_to_migrate

    return images

In [ ]:
def migrate_project(source_trainer, destination_trainer, project_id):
    source_project = source_trainer.get_project(project_id)

    print ("Source Project:", source_project.name)
    print ("Description:", source_project.description)
    print ("Domain:", source_project.settings.domain_id)
    if source_project.settings.classification_type:
        print ("ClassificationType:", source_project.settings.classification_type)
    print("Target Export Platforms:", source_project.settings.target_export_platforms)

    return destination_trainer.create_project(source_project.name, description=source_project.description, domain_id=source_project.settings.domain_id, classification_type=source_project.settings.classification_type, target_export_platforms=source_project.settings.target_export_platforms)

In [ ]:
source_project_id = ''
source_training_key = ''
source_endpoint = ''
destination_training_key = ''
destination_endpoint = ''

source_trainer = CustomVisionTrainingClient(source_training_key, source_endpoint)

destination_trainer = CustomVisionTrainingClient(destination_training_key, endpoint=destination_endpoint)

destination_project = migrate_project(source_trainer, destination_trainer, source_project_id)
tags = migrate_tags(source_trainer, destination_trainer, source_project_id, destination_project.id)
source_images = migrate_images(source_trainer, destination_trainer, source_project_id, destination_project.id, tags)